In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

# Masa total y Radio total de modelos con polítropa maestra

Este código calcula la masa total ($M$), el radio total ($r_b$) y la compacidad ($\mu_b$) de esferas anisótropas relativistas con ecuación de estado polítropa maestra.

## Ecuaciones de estructura

Las ecuaciones de estructura son el resultado de resolver y manipular las ecuaciones de campo de Einstein para una métrica y un tensor de energía dado. En el caso de configuraciones estáticas con simetría esférica y anisotropía en la presión, las ecuaciones de estructura consisten en

\begin{equation}
\label{EcEqHidro}
P^{\prime} = - \left(\rho + P \right) \nu^{\prime} + \frac{2}{r} \left(P_{\perp} - P \right).
\end{equation}

\ref{EcEqHidro}







In [2]:
# Definiendo sistema de ecuaciones
def funciones(xi,y, alpha, n, h, Upsilon, varkappa):
    psi_ , eta_, psi1_, eta1_ = y
    dydxi = [-h*(eta_+ xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - (varkappa**(1 + 1/n)/psi_**(n))) + alpha*(1 - (varkappa/psi_**(n))))/xi/(xi - 2*Upsilon*(n+1)*eta_) /(1 + alpha*n/Upsilon/(n+1)/psi_) 
            ,xi**(2) * psi_**(n)
            ,-(h/xi/(xi - 2*Upsilon*(n+1)*eta_)/(1 + alpha*n/Upsilon/(n+1)/psi_))*((eta1_ + 3*xi**(2)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)) + xi**(3)*(n+1)*Upsilon*psi_**(n)*psi1_*(1 + alpha*n/Upsilon/(n+1)/psi_))*(1 + Upsilon*(psi_ - varkappa**(1 + 1/n)/psi_**(n)) + alpha*(1 - varkappa/psi_**(n))) + (eta_ + xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1+ 1/n)) + alpha*(psi_**(n) - varkappa)))*((n+1)*Upsilon*psi1_*(1 + alpha*n/Upsilon/(n+1)/psi_) - n*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa))*psi1_/psi_**(n+1)) + (eta_ + xi**(3)*(Upsilon*(psi_**(n+1) - varkappa**(1 + 1/n)) + alpha*(psi_**(n) - varkappa)))*(1 + Upsilon*(psi_ - varkappa**(1 + 1/n)/psi_**(n)) + alpha*(1 - varkappa/psi_**(n)))*((2*Upsilon*(n+1)*eta1_ - 1)/(xi - 2*Upsilon*(n+1)*eta_) + alpha*n*psi1_/(1 + alpha*n/Upsilon/(n+1)/psi_)/Upsilon/(n+1)/psi_**(2) - 1/xi))
            ,2*xi*psi_**(n) + n*xi**(2)*psi_**(n-1)*psi1_] 
    return dydxi

In [4]:
c = 2.997*10**(8)          # Velocidad de la luz en S.I.
G = 6.673*10**(-11)        # Constante de gravitación universal en S.I.
Msun = 1.989*10**(30)      # Masa solar en S.I.

In [5]:
C = 0.05
h = 1 - 2*C  # Valor de C para anisotropía

Lista_n = [0.8] #Índice polítropo

rho_c = 1.5*10**(18)

alpha = 0.0 # Constante del término lineal

varkappa = 0.0 # Razón entre densidad en el borde y densidad central

# Sigma = Presión central entre Densidad central
Lista_sigma = [0.2]

In [6]:
for i in range(len(Lista_n)):
        
        n = Lista_n[i]
        
        for j in range(len(Lista_sigma)):
            
            sigma = Lista_sigma[j]
    
            Upsilon = (sigma - alpha*(1 - varkappa))/(1 - varkappa**(1 + 1/n))
        
            K = Upsilon*c**(2)/rho_c**(1/n)      # k**(n) en m**(3)/kg
                
            # Condiciones iniciales
            Psi0 = 1.0
            Eta0 = 0.0
            Psi10 = 0.0
            Eta10 = 0.0
            
            y0 = [Psi0,Eta0,Psi10,Eta10]
            
            def stop_condition(xi,y,alpha, n, h, Upsilon, varkappa):
                return (1/sigma)*(Upsilon*(y[0]**(n+1) - varkappa**(1 + 1/n)) + alpha*(y[0]**(n) - varkappa)) - 10**-15
            
            stop_condition.terminal = True
            
            xi0 = 10**(-15)
            ximax = 10000
            
            xi_span = (xi0,ximax)
           
            soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                                   args=(alpha, n, h, Upsilon, varkappa))
            
            Psi = soluciones.y[0]
            Eta = soluciones.y[1]
            Psi1 = soluciones.y[2]
            Eta1 = soluciones.y[3]
            xi = soluciones.t 
            
            
            M = (c**(2)*(n+1)/(4*np.pi)**(1/3)/G)**(3/2)*(K/c**(2))**(n/2)*sigma**((3-n)/2)*Eta[-1]/Msun  # Masa en unidades de masa solar
                    
            R = (c**(2)*(n+1)/4/np.pi/G)**(1/2)*(K/c**(2))**(n/2)*sigma**((1-n)/2)*xi[-1]/1000
            
            mr = 2*Upsilon*(n+1)*Eta/xi
            
            Densidad = Psi**(n)
            
            print('Masa Gravitacional: ', M, '[M_sun],   ', 'Radio: ', R, ' [km], ', '\u03bc = ', mr[-1])
            print('rho_b: ',varkappa*rho_c,Densidad[-1])
            print('Parámetros: ', 'n = {} ,'.format(Lista_n[i]), '\u03c3 = {}'.format(Lista_sigma[j]))

Masa Gravitacional:  1.7018377785898176 [M_sun],    Radio:  11.237804731104262  [km],  μ =  0.4475577650426109
rho_b:  0.0 2.1544346795233424e-07
Parámetros:  n = 0.8 , σ = 0.2


C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
